In [484]:
import pandas as pd
import numpy as np
import datetime as dt

In [511]:
df=pd.read_excel('Preppin Data Challenge.xlsx')

In [512]:
df=df[~df['Project'].str.contains('total',case=False)].reset_index(drop=True).copy()

In [513]:
dim = df.columns[:2]
meas = df.columns[2:]

In [514]:
#df = df.melt(id_vars=dim)
df = df.set_index(['Name, Age, Area of Work','Project']).stack().reset_index()

In [515]:
df = df.rename(columns={'level_2':'date',0:'hours'}).copy()

In [516]:
df[['name','age']] = df['Name, Age, Area of Work'].str.split(',',expand=True)

In [517]:
df[['age','area_of_work']] = df['age'].str.split(':',expand=True)
df.head()

,"Name, Age, Area of Work",Project,date,hours,name,age,area_of_work
0,"Dan, 28: Client",Client Meetings,2021-02-02,2,Dan,28,Client
1,"Dan, 28: Client",Client Meetings,2021-02-04,1,Dan,28,Client
2,"Dan, 28: Client",Client Meetings,2021-02-08,1.5,Dan,28,Client
3,"Dan, 28: Client",Client Meetings,2021-02-09,0.5,Dan,28,Client
4,"Dan, 28: Client",Client Meetings,2021-02-12,Annual Leave,Dan,28,Client


In [518]:
del df['Name, Age, Area of Work']

In [519]:
df = df[['name', 'age', 'area_of_work','Project', 'date', 'hours']]

In [408]:
# df['hours'].fillna(0,inplace=True)

In [521]:
df = df[df['hours']!='Annual Leave'].copy()

In [522]:
df['hours']=df['hours'].astype('float')
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 176 entries, 0 to 192
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   name          176 non-null    object        
 1   age           176 non-null    object        
 2   area_of_work  176 non-null    object        
 3   Project       176 non-null    object        
 4   date          176 non-null    datetime64[ns]
 5   hours         176 non-null    float64       
dtypes: datetime64[ns](1), float64(1), object(4)
memory usage: 9.6+ KB


In [523]:
total_hrs = df.groupby(['area_of_work','name','date']).sum().reset_index().copy()

In [524]:
hrs_per_person = total_hrs.groupby(['name']).sum().copy()

In [525]:
days_per_person = total_hrs[['name','date']].groupby(['name','date']).count().reset_index()
days_per_person = days_per_person.pivot_table(values='date',index='name',aggfunc='count')

In [526]:
total_hrs['area_of_work'] = total_hrs['area_of_work'].str.strip()


In [527]:
hrs_per_person_less_chat = total_hrs[total_hrs['area_of_work']!='Chats'].reset_index(drop=True)


In [528]:
hrs_less_chat = hrs_per_person_less_chat.groupby(['name','area_of_work']).sum().copy()

In [529]:
hrs_less_chat = hrs_less_chat.pivot_table(values='hours',columns='area_of_work',index='name').copy()

In [530]:
combined = hrs_less_chat.merge(hrs_per_person,on='name')

In [531]:
combined = combined.merge(days_per_person,on='name')

In [533]:
combined['area_of_work'] = 'Client'
combined['work_hours'] = combined['Client']+combined['Special Projects']

In [539]:
combined['pct_of_total'] = combined['Client']/combined['work_hours']

In [537]:
combined['hours_per_day'] = combined['hours']/combined['date']

In [541]:
combined[['area_of_work','pct_of_total','hours_per_day']]

,area_of_work,pct_of_total,hours_per_day
name,,,
Dan,Client,0.750000,8.027778
George,Client,0.812950,8.400000
Sam,Client,0.868852,7.700000
